In [2]:
import pandas as pd
from rpart.RandomForestClassifier import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [5]:
adult = pd.read_csv("data/adult.csv")
X = adult.drop("income", axis=1)
y = adult["income"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)
# Create and train the DecisionTreeClassifier
clf = RandomForestClassifier(
    n_estimators=100, max_depth=3, n_workers=10, metric="gini"
)
clf.fit(X_train, y_train)

# Predict on the test set
y_pred = clf.predict(X_test)



In [6]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.7684512232572422
Classification Report:
              precision    recall  f1-score   support

       <=50K       0.77      1.00      0.87      7429
        >50K       1.00      0.03      0.06      2340

    accuracy                           0.77      9769
   macro avg       0.88      0.52      0.47      9769
weighted avg       0.82      0.77      0.68      9769



In [7]:
from rpart.utils import reduceByKey
data = [(("a", 1),  1), (("b", 1), 2), (("c", 1), 3), ("a", 1, 2), (("b", 1), 4)]


ModuleNotFoundError: No module named 'pyspark.RDD'

{('a', 1, 1): [('a', 1, 1)],
 ('a', 1, 2): [('a', 1, 2)],
 ('b', 1, 2): [('b', 1, 2)],
 ('b', 2, 3): [('b', 2, 3)],
 ('c', 1, 3): [('c', 1, 3)],
 ('c', 3, 1): [('c', 3, 1)]}